# Workload Characterization

## Introduction

### Import relevant modules

In [1]:
# Data Analysis
import pandas as pd

# Data Visualization
from matplotlib import pyplot as plt

# Machine Learning
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering

# Utilities
import numpy as np
# import warnings

In C:\Users\fabio\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\fabio\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\fabio\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\fabio\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\fabio\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mpl

In [2]:
%config InlineBackend.figure_format = 'retina'

### Read data

#### High level parameters

In [20]:
df_high = pd.read_csv('WC_report_client.csv')
df_high.head()

,timeStamp,elapsed,label,responseCode,responseMessage,threadName,dataType,success,failureMessage,bytes,sentBytes,grpThreads,allThreads,URL,Latency,IdleTime,Connect
0,1609257155820,44,HTTP Request - Repubblica Home,200,OK,Thread Group - Fast 2-1,text,True,NaN,387753,135,1,2,http://192.168.56.101/RepubblicaHome.html,41,0,27
1,1609257155820,44,HTTP Request - Reddit Home,200,OK,Thread Group - Slow 1-1,text,True,NaN,911062,131,1,2,http://192.168.56.101/RedditHome.html,41,0,27
2,1609257156192,14,HTTP Request - Reddit Home,200,OK,Thread Group - Slow 1-2,text,True,NaN,911062,131,2,4,http://192.168.56.101/RedditHome.html,3,0,2
3,1609257156206,7,HTTP Request - Repubblica Home,200,OK,Thread Group - Fast 2-2,text,True,NaN,387753,135,2,4,http://192.168.56.101/RepubblicaHome.html,2,0,1
4,1609257156306,3,HTTP Request - Repubblica Page,200,OK,Thread Group - Fast 2-1,text,True,NaN,141098,135,2,4,http://192.168.56.101/RepubblicaPage.html,2,0,0


In [21]:
print('The dataset contains', df_high.shape[1], 'columns and', df_high.shape[0], 'rows')

The dataset contains 17 columns and 7454 rows


In [22]:
df_high['success'].all()

True

#### Low level parameters

In [44]:
df_low = pd.read_csv('WC_report_server.csv', sep='\s+', skiprows=1)
df_low.rename(columns={'r':'procs_r', 'b':'procs_r',
                       'swpd':'memory_swpd', 'free':'memory_free', 'buff':'memory_buff', 'cache':'memory_cache', 
                       'si':'swap_si', 'so':'swap_so', 
                       'bi':'io_bi', 'bo':'io_bo',
                       'in':'system_in', 'cs':'system_cs',
                       'us':'cpu_us', 'sy':'cpu_sy', 'id':'cpu_id', 'wa':'cpu_wa', 'st':'cpu_st'
                      }, inplace=True)
df_low.head()

,procs_r,procs_r,memory_swpd,memory_free,memory_buff,memory_cache,swap_si,swap_so,io_bi,io_bo,system_in,system_cs,cpu_us,cpu_sy,cpu_id,cpu_wa,cpu_st
0,4,0,254148,121724,56628,1332972,4,20,143,217,245,1285,12,2,84,2,0
1,0,0,254148,121716,56632,1333004,0,0,0,72,139,370,6,0,85,9,0
2,0,0,254148,121716,56632,1333008,0,0,0,0,59,100,0,0,100,0,0
3,0,0,254148,121716,56632,1333008,0,0,0,0,70,123,0,0,100,0,0
4,0,0,254148,121716,56632,1333008,0,0,0,0,107,4209,8,3,89,0,0


In [45]:
print('The dataset contains', df_low.shape[1], 'columns and', df_low.shape[0], 'rows')

The dataset contains 17 columns and 310 rows
